# Sort and commute

1. import hdf5 archives
2. decode hamiltonians from hdf5's
3. Sort paulis by size, group into commuting buckets, keep coefficients
    - Learn cirq/of package

In [3]:
import matplotlib.pyplot as plt
import numpy as np

import cirq
import openfermion as of

import math
import os
from typing import Callable, Iterable, List, Set
#from tqdm.notebook import tqdm

1. Import hdf5 archives

In [ ]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile


def download_and_unzip(url, extract_to='.'):
    http_response = urlopen(url)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path=extract_to)

In [8]:
url="https://portal.nersc.gov/cfs/m888/dcamps/hamlib/chemistry/electronic/standard/"
molecules=['O2', 'B2', 'BeH', 'BH', 'CH', 'HF', 'C2', 'OH', 'N2', 'Li2', 'NaLi']


for molecule in molecules:
    if os.path.isfile('./'+molecule+'.hdf5') != True:
        download_and_unzip(url+molecule+'.hdf5.zip')


2. Decode hdf5 hamiltonians, done primarily using code from k-commute paper

In [10]:
import h5py


def read_openfermion_hdf5(fname_hdf5: str, key: str, optype=of.QubitOperator):
    """
    Read any openfermion operator object from HDF5 file at specified key.
    'optype' is the op class, can be of.QubitOperator or of.FermionOperator.
    """

    with h5py.File(fname_hdf5, 'r', libver='latest') as f:
        op = optype(f[key][()].decode("utf-8"))
    return op


def parse_through_hdf5(func):
    """
    Decorator function that iterates through an HDF5 file and performs
    the action specified by ‘ func ‘ on the internal and leaf nodes in the HDF5 file.
    """

    def wrapper (obj, path = '/', key = None) :
        if type(obj) in [h5py._hl.group.Group, h5py._hl.files.File]:
            for ky in obj.keys() :
                func(obj, path, key=ky, leaf = False)
                wrapper(obj = obj[ky], path = path + ky + ',', key = ky)
        elif type (obj) == h5py._hl.dataset.Dataset:
            func(obj, path, key = None, leaf = True)
    return wrapper


def get_hdf5_keys ( fname_hdf5 : str ) :
    """ Get a list of keys to all datasets stored in the HDF5 file .
    Args
    ----
    fname_hdf5 ( str ) : full path where HDF5 file is stored
    """

    all_keys = []
    @parse_through_hdf5
    def action(obj, path = '/', key = None, leaf = False):
        if leaf is True :
            all_keys.append(path)

    with h5py.File(fname_hdf5, 'r') as f:
        action(f['/'])
    return all_keys

In [21]:
molecule_to_read = 'O2'

hamiltonian = read_openfermion_hdf5(
    './'+molecule_to_read+'.hdf5',
    get_hdf5_keys('./'+molecule_to_read+'.hdf5')[0].rstrip(",")
)


openfermion.ops.operators.qubit_operator.QubitOperator

Sort by size

In [101]:

hamiltonian = hamiltonian

terms = sorted(hamiltonian.terms.items(), key=lambda x: abs(x[-1]), reverse=True)


sorted_hamiltonian = [term for term in terms]

sorted_hamiltonian = np.array(sorted_hamiltonian)

list

In [34]:
hamiltonian.terms.keys()


dict_keys([(), ((0, 'X'), (1, 'X'), (2, 'X'), (3, 'Y'), (5, 'Y')), ((0, 'X'), (1, 'X'), (2, 'Z'), (3, 'Y'), (4, 'Y'), (5, 'X')), ((0, 'X'), (1, 'X'), (3, 'X'), (4, 'Y'), (5, 'Y'), (6, 'Z'), (7, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (4, 'Z'), (5, 'Y'), (6, 'Y'), (7, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'X'), (8, 'Y'), (9, 'X'), (11, 'Y'), (12, 'Z'), (13, 'X'), (15, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'X'), (8, 'Y'), (9, 'Y'), (10, 'Z'), (15, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'X'), (8, 'Z'), (9, 'X'), (11, 'Y'), (12, 'Y'), (13, 'X'), (15, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'X'), (8, 'Z'), (9, 'Y'), (10, 'Y'), (15, 'Z')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'Y'), (8, 'X'), (9, 'X'), (11, 'X'), (12, 'Y'), (13, 'Y'), (14, 'Y')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'Y'), (8, 'X'), (9, 'Y'), (10, 'Y'), (11, 'Z'), (13, 'Z'), (14, 'Y')), ((0, 'X'), (1, 'X'), (3, 'X'), (7, 'Y'), (8, 'Y'), (9, 'X'), (11, 'X'), (12, 'X'), (13, 'Y'), (14, 'Y')), ((0, 'X'), (1, 'X'), (3, 'X')